In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle as pkl
import time

In [2]:
data_path = '/labs/hulab/stark_stuff/ppg_ecg_project/data/'
AF_paths = [f'{data_path}/AF_v5/{file}' for file in os.listdir(f'{data_path}/AF_v5/') if file.endswith('.npy') and 'resampled2400' not in file]
NSR_paths = [f'{data_path}/NSR_v5/{file}' for file in os.listdir(f'{data_path}/NSR_v5/') if file.endswith('.npy') and 'resampled2400' not in file]
PVC_paths = [f'{data_path}/PVC_v5/{file}' for file in os.listdir(f'{data_path}/PVC_v5/') if file.endswith('.npy') and 'resampled2400' not in file]

print(len(AF_paths), len(NSR_paths), len(PVC_paths))

4038664 2400994 2118459


In [17]:
time.ctime(os.path.getmtime('/labs/hulab/stark_stuff/ppg_ecg_project/data//AF_v5/DE887834146775714_20140414210546_16154_20140418203549.npy'))

'Wed Jan  4 14:23:39 2023'

In [21]:
for p in tqdm(AF_paths):
    if not ('Jan  3' in time.ctime(os.path.getmtime(p)) or 'Jan  4' in time.ctime(os.path.getmtime(p)) or 'Jan  5' in time.ctime(os.path.getmtime(p))):
        print(p, time.ctime(os.path.getmtime(p)))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4038668/4038668 [00:24<00:00, 162503.66it/s]

/labs/hulab/stark_stuff/ppg_ecg_project/data//AF_v5/train_ECG_resampled2400.npy Thu Feb 16 23:05:35 2023
/labs/hulab/stark_stuff/ppg_ecg_project/data//AF_v5/train_PPG_resampled2400.npy Thu Feb 16 23:07:26 2023
/labs/hulab/stark_stuff/ppg_ecg_project/data//AF_v5/val_ECG_resampled2400.npy Thu Feb 16 19:34:33 2023
/labs/hulab/stark_stuff/ppg_ecg_project/data//AF_v5/val_PPG_resampled2400.npy Thu Feb 16 19:34:43 2023


In [23]:
AF_paths[:1]

['/labs/hulab/stark_stuff/ppg_ecg_project/data//AF_v5/DE609645727247408_20120805020952_2576_20120807132135.npy']

In [24]:
# for path in tqdm(AF_paths):
    
#     sig = np.load(path)
#     ECG = sig[:, 0]
#     PPG = sig[:, 1]
    
#     PPG = (PPG - np.min(PPG)) / (np.max(PPG) - np.min(PPG))
#     ECG = (ECG - np.min(ECG)) / (np.max(ECG) - np.min(ECG))
    
#     cat = np.concatenate((ECG.reshape(ECG.shape[0], 1), PPG.reshape(PPG.shape[0], 1)), axis=1)
#     np.save(path, cat)


In [3]:
AF_patients = set()

for path in tqdm(AF_paths):
    patient_id = path.split('/')[-1].split('_')[0]
    AF_patients.add(patient_id)

NSR_patients = set()
for path in tqdm(NSR_paths):
    patient_id = path.split('/')[-1].split('_')[0]
    NSR_patients.add(patient_id)

PVC_patients = set()
for path in tqdm(PVC_paths):
    patient_id = path.split('/')[-1].split('_')[0]
    PVC_patients.add(patient_id)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2118459/2118459 [00:01<00:00, 1346178.57it/s]


In [4]:
patients = list(set(list(AF_patients) + list(NSR_patients) + list(PVC_patients)))
pseudo_labels = []
for p in patients:
    if p in AF_patients: # AF fisrt since AF set is smallest
        pseudo_labels.append(0)
    elif p in PVC_patients:
        pseudo_labels.append(1)
    elif p in NSR_patients:
        pseudo_labels.append(2)
        
len(patients)

20048

In [5]:
from sklearn.model_selection import train_test_split
patient_train, patient_val, y_train, y_val = train_test_split(patients, pseudo_labels, test_size=0.33, random_state=42)

In [6]:
print(len(patient_train), len(patient_val))

13432 6616


In [8]:
len(set(patient_train).intersection(set(patient_val)))

0

In [11]:
np.unique(pseudo_labels, return_counts=True)

(array([0, 1, 2]), array([ 2734, 16268,  1046]))

In [9]:
np.unique(y_train, return_counts=True)

(array([0, 1, 2]), array([ 1839, 10895,   698]))

In [10]:
np.unique(y_val, return_counts=True)

(array([0, 1, 2]), array([ 895, 5373,  348]))

In [14]:
def split_paths(paths, set_name):
    
    train_paths = []
    val_paths = []

    for path in tqdm(paths):
        patient_id = path.split('/')[-1].split('_')[0]

        if patient_id in patient_train: train_paths.append(path)
        elif patient_id in patient_val: val_paths.append(path)
        else: print('Error', patient_id)
    print(len(train_paths), len(val_paths))
    pkl.dump(train_paths, open(f'{data_path}/{set_name}_v5/split_train.pkl', 'wb'))
    pkl.dump(val_paths, open(f'{data_path}/{set_name}_v5/split_val.pkl', 'wb'))    

In [15]:
split_paths(AF_paths, 'AF')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4038664/4038664 [10:52<00:00, 6191.72it/s]


2757889 1280775


In [16]:
split_paths(NSR_paths, 'NSR')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2400994/2400994 [06:34<00:00, 6093.13it/s]


1603176 797818


In [17]:
split_paths(PVC_paths, 'PVC')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2118459/2118459 [05:47<00:00, 6099.51it/s]


1411158 707301
